In [1]:
FEATURE_PER_DAY = 18
WINDOW_SIZE = 9
FEATURE_NUM = 18 * 9

In [2]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import random

df = pd.read_csv("competitions/ml-2018spring-hw1/train.csv", encoding="big5")

for row in np.arange(10, len(df), FEATURE_PER_DAY):
    df.iloc[row, 3:] = pd.to_numeric(df.iloc[row, 3:], errors="coerce")

df.fillna(0, inplace=True)

In [3]:
x_data = []
y_data = []

for i in np.arange(0, len(df), FEATURE_PER_DAY):
    arr = np.array(df.iloc[i:i+FEATURE_PER_DAY, 3:3+WINDOW_SIZE], dtype=float)
    arr = np.insert(arr, 0, 1) # x0 for bias
    x_data.append(arr)
    y_data.append(np.array(df.iloc[i+9, 3+WINDOW_SIZE], dtype=float))

In [4]:
x_data[0]

array([  1.  ,  14.  ,  13.  ,  13.  ,  12.  ,  12.  ,  12.  ,  12.  ,
        12.  ,  14.  ,   1.9 ,   1.9 ,   2.1 ,   2.1 ,   2.2 ,   2.2 ,
         2.2 ,   2.1 ,   2.  ,   1.16,   1.  ,   1.09,   1.18,   1.01,
         1.  ,   1.15,   1.33,   1.36,   0.7 ,   0.59,   0.66,   0.65,
         0.52,   0.5 ,   0.58,   0.66,   0.61,  17.  ,   8.8 ,  14.  ,
        33.  ,  26.  ,  21.  ,  34.  ,  38.  ,  37.  ,  31.  ,  29.  ,
        26.  ,  25.  ,  23.  ,  21.  ,  20.  ,  22.  ,  30.  ,  47.  ,
        38.  ,  41.  ,  57.  ,  49.  ,  41.  ,  54.  ,  61.  ,  67.  ,
         2.4 ,   3.  ,   2.6 ,   2.3 ,   2.1 ,   2.3 ,   2.5 ,   3.4 ,
         7.1 ,  91.  , 104.  ,  87.  ,  91.  ,  83.  ,  76.  ,  73.  ,
        66.  ,  88.  ,  58.  ,  59.  ,  57.  ,  59.  ,  64.  ,  64.  ,
        60.  ,  55.  ,  54.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,  79.  ,  79.  ,  80.  ,  83.  ,
        83.  ,  83.  ,  82.  ,  83.  ,  79.  ,   3.4 ,   3.6 ,   2.7 ,
      

In [5]:
# Step 1: Model
# Loss function L(f) = sum of (y - f)^2
theta = np.array([0.0] * (FEATURE_NUM + 1)) # all parameters init to 1
# for i in range(len(theta)):
#     theta[i] = random.randint(0, 99)
lr = 0.001
iteration = 10000

lr_ada = np.zeros(FEATURE_NUM + 1)

In [ ]:
theta = np.load('theta.npy')

In [ ]:
from tqdm import tqdm

for i in tqdm(range(iteration)):
    grad = np.zeros(FEATURE_NUM + 1)
    
    for n in range(len(x_data)):
        dot = np.dot(theta, x_data[n])
        for f in range(len(grad)):
            grad[f] = grad[f] - 2.0 * (y_data[n] - dot) * x_data[n][f]
    
    for f in range(len(grad)):
        lr_ada[f] = lr_ada[f] + grad[f] ** 2

    for index in range(len(theta)):
        theta[index] = theta[index] - lr / np.sqrt(lr_ada[index]) * grad[index]

 99%|█████████▊| 9853/10000 [12:40<00:11, 12.96it/s]

In [ ]:
print(grad)

In [ ]:
def computeCost(x_data, y_data, theta):
    cost = 0.0
    for i in range(len(x_data)):
        cost += (np.dot(theta, x_data[i]) - y_data[i]) ** 2
    return cost

In [ ]:
record = [16895.726658446325]

In [ ]:
record.append(computeCost(x_data, y_data, theta))
print(record)

In [ ]:
for i in range(len(x_data)):
    print(np.dot(theta, x_data[i]), y_data[i])

In [ ]:
np.save('theta', theta)

In [ ]:
dt = pd.read_csv("competitions/ml-2018spring-hw1/test.csv", header=None, encoding="big5")

for row in np.arange(10, len(dt), FEATURE_PER_DAY):
    dt.iloc[row, 2:] = pd.to_numeric(dt.iloc[row, 2:], errors="coerce")
    
dt.fillna(0, inplace=True)

In [ ]:
x_test = []
y_test = []

for i in np.arange(0, len(dt), FEATURE_PER_DAY):
    arr = np.array(dt.iloc[i:i+FEATURE_PER_DAY, 2:2+WINDOW_SIZE], dtype=float)
    arr = np.insert(arr, 0, 1) # x0 for bias
    x_test.append(arr)
    
for i in range(len(x_test)):
    y_test.append(np.dot(theta, x_test[i]))

In [ ]:
for i in range(len(y_test)):
    if (y_test[i] < 0):
        y_test[i] = 0

In [ ]:
arr = [["id_" + str(i), y_test[i]] for i in range(len(y_test))]
dw = pd.DataFrame(arr, columns = ["id", "value"])
dw.to_csv("output.csv", index=False)